In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn import linear_model
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier, MLPRegressor
import sklearn.metrics as metrics
from sklearn.ensemble import StackingRegressor, StackingClassifier
import xgboost as xgb
import pandas as pd
import time
import multiprocess
from ensemble_func import ps_function,ps_predict,gcompu_function,gcompu_predict

# loading data

In [3]:
abcd = pd.read_csv("ABCD5.0_flwup2_allnih.csv")
#from DR_function import dr,dr_boot
import pyreadr
robjects = pyreadr.read_r('ABCD5.0_varname_allnih.RData')
# Convert the R object to a Pandas DataFrame
covbase = robjects['covbase'].to_numpy().ravel()
output1 = robjects['outtp12_1'].to_numpy().ravel()
output2 = robjects['outtp12_2'].to_numpy().ravel()
covtp1 = robjects['covtp1'].to_numpy().ravel()
covtp2 = robjects['covtp2'].to_numpy().ravel()
trttp1 = robjects['trttp1'].to_numpy().ravel()
trttp2 = robjects['trttp2'].to_numpy().ravel()

# coding dummy variables for categorical variables (only consider race and income, education need to be discuss)
cat_cols = abcd.select_dtypes(include="object").columns.tolist()[2:]
dummy_df = pd.get_dummies(abcd[cat_cols],prefix_sep="",drop_first=True)
dummy_df.columns = dummy_df.columns.str.replace('_r', '', regex=True)
abcd_encoded = pd.concat([abcd.drop(cat_cols,axis=1),dummy_df],axis=1)
covbase = np.concatenate([[covbase[0]],covbase[4:],dummy_df.columns])
#abcd_encoded = abcd


# Estimating PS

In [ ]:
# with common propensity score
ps0feature = np.concatenate([covbase,covtp1])
ps1feature = np.concatenate([covbase,covtp1,covtp2, output1,trttp1])

baseps0_tr,baseps0_ts,cls_stack0_tr,cls_stack0_ts = ps_function(abcd_encoded,ps0feature,trttp1)
enps0 = ps_predict(abcd_encoded,ps0feature,baseps0_tr,baseps0_ts,cls_stack0_tr,cls_stack0_ts)

baseps1_tr,baseps1_ts,cls_stack1_tr,cls_stack1_ts = ps_function(abcd_encoded,ps1feature,trttp2,l1=True,mlp=True)
enps1 = ps_predict(abcd_encoded,ps1feature,baseps1_tr,baseps1_ts,cls_stack1_tr,cls_stack1_ts)
IPW = (abcd_encoded[trttp1].values.ravel()/enps0+(1-abcd_encoded[trttp1].values.ravel())/(1-enps0))*(abcd_encoded[trttp2].values.ravel()/enps1+(1-abcd_encoded[trttp2].values.ravel())/(1-enps1))
msm_lm = linear_model.LinearRegression()
phi_msm_sum = []; phi_msm_ratio = []; phi_msm1 = []; phi_msm2 = []
for out2 in output2:
 # print(out2)
 msm_lm.fit(abcd_encoded[[trttp1[0],trttp2[0]]],abcd_encoded[out2],sample_weight=IPW)
 phi_msm_sum.append((np.sum(msm_lm.coef_))); phi_msm1.append(msm_lm.coef_[0]); phi_msm2.append(msm_lm.coef_[1])
 phi_msm_ratio.append(phi_msm_sum/(phi_msm_sum+msm_lm.intercept_))

# Estimating Effects

In [ ]:
from dr_estimation_func import dr_abcd_boot
start_time = time.time()
#np.random.seed(233)
with multiprocess.Pool(processes=30) as pool:
    #np.random.seed(233)
    result_abcd_boot = pool.starmap(dr_abcd_boot, zip(output1,output2))
    pool.close(); pool.join()

end_time = time.time()
(end_time-start_time)